In [1]:
import numpy as np

# Load the .npz file
data = np.load('zipped_vids_1000.npz')

# Convert to a Python dictionary
data_vids= {key: data[key] for key in data.files}

In [3]:
import numpy as np

# Load the .npz file
data = np.load('zipped_vids_3.npz')

# Convert to a Python dictionary
data_vids= {key: data[key] for key in data.files}


import json

with open("s1_proc_align.json", 'r') as f:
    target = json.load(f)

# Extracting specific key-value pairs
selected_keys = list(data_vids.keys())
selected_target = {k: target[k] for k in selected_keys if k in target}
filtered_data = {k: v for k, v in data.items() if k != '.DS_Store'}
filtered_target = {k: v for k, v in selected_target.items() if k != '.DS_Store'}

print(selected_target);

{'bbaz6p': [2, 9, 14, 39, 2, 12, 21, 5, 39, 1, 20, 39, 26, 39, 19, 9, 24, 39, 16, 12, 5, 1, 19, 5], 'bbaszn': [2, 9, 14, 39, 2, 12, 21, 5, 39, 1, 20, 39, 19, 39, 26, 5, 18, 15, 39, 14, 15, 23], 'bbaz4n': [2, 9, 14, 39, 2, 12, 21, 5, 39, 1, 20, 39, 26, 39, 6, 15, 21, 18, 39, 14, 15, 23], 'bbas2p': [2, 9, 14, 39, 2, 12, 21, 5, 39, 1, 20, 39, 19, 39, 20, 23, 15, 39, 16, 12, 5, 1, 19, 5], 'bbaz5s': [2, 9, 14, 39, 2, 12, 21, 5, 39, 1, 20, 39, 26, 39, 6, 9, 22, 5, 39, 19, 15, 15, 14], 'bbbf9a': [2, 9, 14, 39, 2, 12, 21, 5, 39, 2, 25, 39, 6, 39, 14, 9, 14, 5, 39, 1, 7, 1, 9, 14], 'bbbf7s': [2, 9, 14, 39, 2, 12, 21, 5, 39, 2, 25, 39, 6, 39, 19, 5, 22, 5, 14, 39, 19, 15, 15, 14], 'bbaf4p': [2, 9, 14, 39, 2, 12, 21, 5, 39, 1, 20, 39, 6, 39, 6, 15, 21, 18, 39, 16, 12, 5, 1, 19, 5], 'bbaf2n': [2, 9, 14, 39, 2, 12, 21, 5, 39, 1, 20, 39, 6, 39, 20, 23, 15, 39, 14, 15, 23], 'bbas3a': [2, 9, 14, 39, 2, 12, 21, 5, 39, 1, 20, 39, 19, 39, 20, 8, 18, 5, 5, 39, 1, 7, 1, 9, 14], 'bbal9a': [2, 9, 14, 39, 2, 

In [4]:
print(len(filtered_data['bbaz6p']))
print(len(filtered_data['bbaz6p'][0]))
print(len(filtered_data['bbaz6p'][0][0]))
print(len(filtered_data['bbaz6p'][0][0][0]))

75
30
70
1


In [5]:
import tensorflow as tf

In [6]:
def create_tf_dataset(data, target, frame_count=75, max_token_length=None):
    # Determine the maximum length of the token sequences if not provided
    if max_token_length is None:
        max_token_length = max(len(t) for t in target.values())

    # Initialize lists for padded video data and targets
    video_data_padded = []
    target_padded = []

    # Iterate through each video and pad frames and token sequences
    for key in data.keys():
        # Pad video frames to uniform length
        padding_frames = frame_count - len(data[key])
        if padding_frames > 0:
            last_frame = data[key][-1]  # Extract the last frame
            padded_video_frames = data[key] + [last_frame] * padding_frames  # Pad with the last frame
        else:
            padded_video_frames = data[key][:frame_count]  # Truncate frames if longer than frame_count

        # Convert to tensor and add to list
        video_tensor = tf.convert_to_tensor(padded_video_frames, dtype=tf.float32)
        video_data_padded.append(video_tensor)

        # Pad token sequences
        padded_token = target[key] + [0] * (max_token_length - len(target[key]))
        target_padded.append(padded_token)

    # Create datasets from padded data
    video_dataset = tf.data.Dataset.from_tensor_slices(video_data_padded)
    token_dataset = tf.data.Dataset.from_tensor_slices(target_padded)

    # Zip datasets to pair videos with their corresponding tokens
    dataset = tf.data.Dataset.zip((video_dataset, token_dataset))

    # Batch dataset
    dataset = dataset.batch(2)

    return dataset

# Assuming 'data' and 'selected_target' are defined earlier in your code
dataset = create_tf_dataset(filtered_data, selected_target)
dataset = dataset.prefetch(tf.data.AUTOTUNE)

# Splitting dataset into training and testing
train_dataset = dataset.take(800)  # Adjust according to your dataset size
test_dataset = dataset.skip(800)


In [22]:
train_dataset.as_numpy_iterator()

In [18]:
train_dataset

<TakeDataset element_spec=(TensorSpec(shape=(None, 75, 30, 70, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 26), dtype=tf.int32, name=None))>

In [8]:
test_dataset

<SkipDataset element_spec=(TensorSpec(shape=(None, 75, 30, 70, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 26), dtype=tf.int32, name=None))>

In [9]:
for key in filtered_data.keys():
    video_tensor = tf.convert_to_tensor(filtered_data[key], dtype=tf.float32)
    print(key, "tensor shape:", video_tensor.shape)

bbaz6p tensor shape: (75, 30, 70, 1)
bbaszn tensor shape: (75, 30, 70, 1)
bbaz4n tensor shape: (75, 30, 70, 1)
bbas2p tensor shape: (75, 30, 70, 1)
bbaz5s tensor shape: (75, 30, 70, 1)
bbbf9a tensor shape: (75, 30, 70, 1)
bbbf7s tensor shape: (75, 30, 70, 1)
bbaf4p tensor shape: (75, 30, 70, 1)
bbaf2n tensor shape: (75, 30, 70, 1)
bbas3a tensor shape: (75, 30, 70, 1)
bbal9a tensor shape: (75, 30, 70, 1)
bbaz7a tensor shape: (75, 30, 70, 1)
bbaf3s tensor shape: (75, 30, 70, 1)
bbas1s tensor shape: (75, 30, 70, 1)
bbal8p tensor shape: (75, 30, 70, 1)
bbal7s tensor shape: (75, 30, 70, 1)
bbbf8p tensor shape: (75, 30, 70, 1)
bbal6n tensor shape: (75, 30, 70, 1)
bbaf5a tensor shape: (75, 30, 70, 1)
bbbf6n tensor shape: (75, 30, 70, 1)


In [10]:
test_dataset

<SkipDataset element_spec=(TensorSpec(shape=(None, 75, 30, 70, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 26), dtype=tf.int32, name=None))>

In [11]:
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv3D, Activation, MaxPool3D, TimeDistributed, Flatten, Bidirectional, LSTM, Dropout, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.optimizers import Adam


def initiate_model():
    model = Sequential()
    model.add(Conv3D(128, (3, 3, 3), input_shape=(75, 30, 70, 1), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPool3D((1,2,2)))

    model.add(Conv3D(256, 3, padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPool3D((1,2,2)))

    model.add(Conv3D(75, 3, padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPool3D((1,2,2)))

    model.add(TimeDistributed(Flatten()))

    model.add(Bidirectional(LSTM(128, kernel_initializer='Orthogonal', return_sequences=True)))
    model.add(Dropout(.5))

    model.add(Bidirectional(LSTM(128, kernel_initializer='Orthogonal', return_sequences=True)))
    model.add(Dropout(.5))

    model.add(Dense(char_to_num.vocabulary_size()+1, kernel_initializer='he_normal', activation='softmax'))
    
    return model

In [12]:
vocab = [x for x in "abcdefghijklmnopqrstuvwxyz'?!123456789 "]
char_to_num = tf.keras.layers.StringLookup(vocabulary=vocab, oov_token="")
num_to_char = tf.keras.layers.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), oov_token="", invert=True
)

print(
    f"The vocabulary is: {char_to_num.get_vocabulary()} "
    f"(size ={char_to_num.vocabulary_size()})"
)

The vocabulary is: ['', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'", '?', '!', '1', '2', '3', '4', '5', '6', '7', '8', '9', ' '] (size =40)


In [23]:
import math
# Function for compiling the model
def compile_model(model):
    model.compile(optimizer=Adam(learning_rate=0.0001), loss=CTCLoss)
    return model

# Define the CTC loss function
def CTCLoss(y_true, y_pred):
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

    loss = tf.keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    return loss

# Define learning rate scheduler
def scheduler(epoch, lr):
    if epoch < 30:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

# Example producing callback
class ProduceExample(tf.keras.callbacks.Callback):
    def __init__(self, dataset) -> None:
        self.dataset = dataset
        self.iterator = iter(dataset)

    def on_epoch_end(self, epoch, logs=None) -> None:
        data = next(self.iterator, None)
        if data is None:  # This would only happen if the dataset is not repeating
            self.iterator = iter(self.dataset)
            data = next(self.iterator, None)
        if data is not None:
            yhat = self.model.predict(data[0])
            decoded = tf.keras.backend.ctc_decode(yhat, [75, 75], greedy=False)[0][0].numpy()
            for x in range(len(yhat)):           
                print('Original:', tf.strings.reduce_join(num_to_char(data[1][x])).numpy().decode('utf-8'))
                print('Prediction:', tf.strings.reduce_join(num_to_char(decoded[x])).numpy().decode('utf-8'))
                print('~'*100)

model = initiate_model()
model = compile_model(model)



# Callbacks
checkpoint_callback = ModelCheckpoint(os.path.join('models','checkpoint'), monitor='loss', save_freq="epoch", save_weights_only=True)
schedule_callback = LearningRateScheduler(scheduler)
example_callback = ProduceExample(test_dataset)

# Fit the model
history = model.fit(train_dataset, validation_data=test_dataset,epochs=100,callbacks=[checkpoint_callback, schedule_callback, example_callback])


Epoch 1/100
 8/10 [=======================>......] - ETA: 9s - loss: 164.9930 

KeyboardInterrupt: 

In [ ]:
#taking 1 minute per eopch when run locally